# PFRA 

update description

### _Development Notebook_
--- 

Tests using scrapbook

In [3]:
import sys;sys.path.append('../')

In [7]:
import gdal
from hecrasio.core import *
from hecrasio.qaqc import *
from hecrasio.s3tools import *
import scrapbook as sb
import pandas as pd
import json
import shutil
from collections import ChainMap

In [5]:
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 1000)

In [8]:
def s3_nbs(bucket:str, prefix:str, nameSelector:str='', fileformat:str='.ipynb') -> list:
    """
    Lists notebooks on S3 when provided with the bucket, object prefix, and
    file format. The default fileformat is IPython (i.e. Jupyter) Notebooks.
    """
    s3_client = boto3.client('s3')
    keys = s3_client.list_objects_v2(Bucket=bucket, Prefix=prefix)
    keysList = [keys]
    pathsList = []
    
    while keys['IsTruncated'] is True:
        keys = s3_client.list_objects_v2(Bucket=bucket, Prefix=prefix,
                                         ContinuationToken=keys['NextContinuationToken'])
        keysList.append(keys)
    for key in keysList:
        key_matches = [elem['Key'] for elem in key['Contents'] if elem['Key'].find('{}'.format(nameSelector))>=0 and elem['Key'].endswith(fileformat)]
        paths = ['s3://{0}/{1}'.format(bucket, key) for  key in key_matches]
        pathsList.extend(paths)

    return pathsList

In [9]:
help(s3nbs)

NameError: name 's3nbs' is not defined

In [4]:
# Read many notebooks, central location
books = sb.read_notebooks('s3://pfra/DC/F01/NBR/nbs')

ClientError: An error occurred (AccessDenied) when calling the ListObjectsV2 operation: Access Denied

In [11]:
# Read many notebooks, diff locations

In [21]:
#### make_qaqc_table()

In [12]:
# Create a unified results dictionary
results_dict = {}
for tup in list(books.items()):
    nb, results = tup
    result_dict = dict(ChainMap(*[list(json.loads(scrap).values())[0] for scrap in results.scraps]))
    results_dict[nb] = result_dict

In [28]:
# Pull results into a large review table
results_table = pd.DataFrame.from_dict(results_dict).T
results_table.head(3)

,Computation Time Step Count To Double,Computation Time Courant Method,2D Eddy Viscosity Transverse Mixing Coefficient,2D Coriolis,Vol Accounting Cell Start,2D Boundary Condition Ramp Up Fraction,Computation Time Total,1D2D MaxIter,Vol Accounting Groundwater,Plan Title,2D Only,Pardiso Solver,2D Initial Conditions Ramp Up Time (hrs),2D Theta,Vol Accounting Lat Hydro,HDF Chunk Size,Computation Time Step Base,Computation Time Step Min Courant,Computation Time Step Max Doubling,2D Boundary Condition Volume Check,2D Maximum Iterations,Computation Time DSS,1D2D Minimum Flow Tolerance,Geometry Title,HDF Write Warmup,Simulation Start Time,2D Number of Time Slices,Vol Accounting Error Percentage,1D2D Flow Tolerance,Flow Title,Solution,2D Names,2D Theta Warmup,Flow Filename,1D Methodology,Computation Time Step Max Halving,HDF Flush Buffer,Maximum WSEL Error,Vol Accounting Error,Plan Name,Vol Accounting DS Outflow,Time Window,Computation Time Step Max Courant,2D Cores,1D Cores,Vol Accounting,Vol Accounting Cell End,HDF Fixed Rows,HDF Write Time Slices,HDF Write Face Node Velocities,HDF Spatial Parts,"Vol Accounting LS, Pump Div",Plan ShortID,Vol Accounting US Inflow,Run Time Window,Time Stamp Solution Went Unstable,HDF Compression,Geometry Filename,Plan Filename,2D Water Surface Tolerance,Time Solution Went Unstable,Simulation End Time,2D Latitude for Coriolis,2D Volume Tolerance,HDF Use Max Rows,1D2D WS Tolerance,Base Output Interval,2D Equation Set,Vol Accounting 2D Bdry
DC_F01_NBR_E0005,2,Representative Length/Velocity,[3.402823466e+38],False,84515.9,[0.1000000015],11:28:49,0,0,test_scaling,True,False,[3.402823466e+38],[1.0],0,1,30SEC,0.5,2,[False],[20],00:00:00,1,D01,False,01May2000 10:00:0,[1],0.00427186,0.1,E0005,Unsteady Finished Successfully,[D01],[1.0],DC_F01_NBR.u01,Finite Difference,1,False,0,-43.0278,test_scaling,0,01May2000 10:00:00 to 05May2000 10:00:0,2,0,0,Volume Accounting in Acre Feet,102312,1,False,False,1,0,test_scaling,0,12AUG2019 20:46:47 to 13AUG2019 08:15:35,Not Applicable,1,DC_F01_NBR.g01,DC_F01_NBR.p01,[0.0099999998],00:00:00,05May2000 10:00:0,[3.402823466e+38],[0.0099999998],0,0.01,15MIN,[Dynamic Wave],0
DC_F01_NBR_E0006,2,Representative Length/Velocity,[3.402823466e+38],False,84515.9,[0.1000000015],11:02:30,0,0,test_scaling,True,False,[3.402823466e+38],[1.0],0,1,30SEC,0.5,2,[False],[20],00:00:01,1,D01,False,01May2000 10:00:0,[1],0.00762016,0.1,E0006,Unsteady Finished Successfully,[D01],[1.0],DC_F01_NBR.u01,Finite Difference,1,False,0,73.0143,test_scaling,0,01May2000 10:00:00 to 05May2000 10:00:0,2,0,0,Volume Accounting in Acre Feet,99545.4,1,False,False,1,0,test_scaling,0,12AUG2019 20:46:51 to 13AUG2019 07:49:19,Not Applicable,1,DC_F01_NBR.g01,DC_F01_NBR.p01,[0.0099999998],00:00:00,05May2000 10:00:0,[3.402823466e+38],[0.0099999998],0,0.01,15MIN,[Dynamic Wave],0


In [22]:
# Function?
# Create a summary table
df = pd.DataFrame(columns=['Unique_Values', 'Warnings', 'Offending_Nbs'])
df['Result_Attribute'] = pd.Index(list(results_table.columns))
df.set_index('Result_Attribute', drop=True, inplace=True)

rows to include in unique table:
    Base Output Interval
    2D Equation Set
    2D Volume Tolerance
    2D Latitude for Coriolis
    2D Water Surface Tolerance
    Solution
    
if len(unique) == 1 for all, ignore: return all tables  identical.

In [26]:
df_w_uniques = identify_unique_values(df, results_table)
df_w_uniques

,Unique_Values,Warnings,Offending_Nbs
Result_Attribute,,,
Computation Time Step Count To Double,[2],NaN,NaN
Computation Time Courant Method,[Representative Length/Velocity],NaN,NaN
2D Eddy Viscosity Transverse Mixing Coefficient,[3.402823466e+38],NaN,NaN
2D Coriolis,[False],NaN,NaN
Vol Accounting Cell Start,[84515.9453125],NaN,NaN
2D Boundary Condition Ramp Up Fraction,[0.1000000015],NaN,NaN
Computation Time Total,"[11:28:49, 11:02:30]",NaN,NaN
1D2D MaxIter,[0],NaN,NaN
Vol Accounting Groundwater,[0.0],NaN,NaN


Threshold tests: return a list enumerating what's going on

Velocity Test failures:

Simulation #of Cells Exceeding {threshold}  Max Velocity (ft/s)

nb1        500                              2,000
nb1        100                              3,000
...


Run Completed Test:

Simulation Status
nb3          Failed

In [17]:
# Example warnings
for i in df_w_uniques.index:
    values = df_w_uniques.loc[i][0]
    if i == '1D Cores':
        validate_by_threshold(df_w_uniques, i, values, 0, results_table)
    elif i == 'Vol Accounting Error':
        validate_by_threshold(df_w_uniques, i, values, 30, results_table)
    elif i == 'Vol Accounting Error Percentage':
        validate_by_threshold(df_w_uniques, i, values, 0.001, results_table)

df_w_uniques.head(5)

,Unique_Values,Warnings,Offending_Nbs
Result_Attribute,,,
Computation Time Step Count To Double,[2],NaN,NaN
Computation Time Courant Method,[Representative Length/Velocity],NaN,NaN
2D Eddy Viscosity Transverse Mixing Coefficient,[3.402823466e+38],NaN,NaN
2D Coriolis,[False],NaN,NaN
Vol Accounting Cell Start,[84515.9453125],NaN,NaN


### Possible evaluations
- Check output interval is the desired amount
- Check if solution populated with anything other than 'Unstead Finished Successfully'
- Check flow, geometry, and plan filenames have the same base

In [ ]:
### Checks strings as the same

In [29]:
def validate_by_equivalency(pd_df:pd.DataFrame, attr:str, values:list, threshold:any, results_df: pd.DataFrame):
    pd_df.loc[attr]['Warnings'] = 'WARNING' if any([value > threshold for value in values]) else 'PASS'
    pd_df.loc[attr]['Offending_Nbs'] = [results_df.index[i] for i, value in enumerate(list(results_df[attr])) if value > threshold]

In [31]:
def validate_by_text(pd_df:pd.DataFrame, attr:str, values:list, regex: str, results_df: pd.DataFrame):
    pd_df.loc[attr]['Warnings'] = 'WARNING' if any([value != regex for value in values]) else 'PASS'
    pd_df.loc[attr]['Offending_Nbs'] = [results_df.index[i] for i, value in enumerate(list(results_df[attr])) if value != regex]

In [ ]:
# Basic logic to check equivalency
interval_step = 15
interval_units = 'MIN'
intervals = [interval[:-3] for interval in test_df.loc['Base Output Interval'][0]]
units = [interval[-3:] for interval in test_df.loc['Base Output Interval'][0]]
'WARNING' if any([interval != str(interval_step) for interval in intervals]) else 'PASS'
'WARNING' if any([unit != str(interval_units) for unit in units]) else 'PASS'

# This needs to include something to evaluate these together...

In [ ]:
# Impliment validate by text solution
for i in test_df.index:
    values = test_df.loc[i][0]
    if i == 'Solution':
        validate_by_text(test_df, i, values, 'Unsteady Finished Successfully', results_table)

In [33]:
# A number of these evaluations need to be done on the results table

# Basic logic for checking flow, geometry, and plan names match
for i in results_table.index:
    ff = results_table.loc[i]['Flow Filename'].split('.')[0]
    gf = results_table.loc[i]['Geometry Filename'].split('.')[0]
    pf = results_table.loc[i]['Plan Filename'].split('.')[0]
    print('WARNING') if any([name != ff for name in [ff, gf, pf]]) else print('PASS')

PASS
PASS


In [ ]:
# Could develop results table and the summary evaluator...this ends up being twice the validation


# END